In [2]:
import streamlit as st
import time
import csv
import random
import paho.mqtt.client as mqtt
from threading import Thread
import pandas as pd
import os

In [ ]:

# Simulated GPIO library for non-hardware environment
class MockGPIO:
    OUTPUT = 'OUTPUT'
    HIGH = 1
    LOW = 0
    
    def __init__(self):
        self.pins = {}
    
    def setmode(self, mode):
        print(f"Set mode to {mode}")
    
    def setup(self, pin, mode):
        if pin not in self.pins:
            self.pins[pin] = {'mode': mode, 'state': self.LOW}
            print(f"Setup pin {pin} as {mode}")
    
    def output(self, pin, state):
        if pin not in self.pins:
            self.setup(pin, self.OUTPUT)
        self.pins[pin]['state'] = state
        st.session_state[f"{pin}_state"] = state
        print(f"Pin {pin} set to {'HIGH' if state == self.HIGH else 'LOW'}")
    
    def input(self, pin):
        if pin not in self.pins:
            self.setup(pin, self.OUTPUT)
        return random.uniform(20.0, 30.0)  # Simulated temperature

GPIO = MockGPIO()

# MQTT setup
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC_CONTROL = "smart_home/control"
MQTT_TOPIC_SENSOR = "smart_home/sensor"

# Device configuration
devices = {
    'led1': {'pin': 18, 'state': GPIO.LOW, 'name': 'Room 1 Light'},
    'led2': {'pin': 19, 'state': GPIO.LOW, 'name': 'Room 2 Light'},
    'fan': {'pin': 23, 'state': GPIO.LOW, 'name': 'Living Room Fan'}
}

# Data logging setup
LOG_FILE = "sensor_data.csv"
ACTION_LOG_FILE = "action_log.csv"

def init_csv():
    if not os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Timestamp', 'Temperature'])
    if not os.path.exists(ACTION_LOG_FILE):
        with open(ACTION_LOG_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Timestamp', 'Device', 'Action'])

# MQTT callbacks
def on_connect(client, userdata, flags, rc):
    print(f"Connected to MQTT broker with code {rc}")
    client.subscribe(MQTT_TOPIC_CONTROL)

def on_message(client, userdata, msg):
    payload = msg.payload.decode()
    print(f"Received MQTT message: {payload}")
    try:
        device, state = payload.split(':')
        state = GPIO.HIGH if state == 'ON' else GPIO.LOW
        if device in devices:
            devices[device]['state'] = state
            GPIO.output(devices[device]['pin'], state)
            log_action(device, 'ON' if state == GPIO.HIGH else 'OFF')
    except Exception as e:
        print(f"Error processing MQTT message: {e}")

# MQTT client setup
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

def start_mqtt():
    try:
        mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
        mqtt_client.loop_start()
    except Exception as e:
        print(f"MQTT connection failed: {e}")

# Log actions
def log_action(device, action):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(ACTION_LOG_FILE, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([timestamp, device, action])
    except Exception as e:
        print(f"Error logging action: {e}")

# Simulate sensor reading and logging
def sensor_loop():
    init_csv()
    while True:
        try:
            temp = GPIO.input(4)  # Simulated temperature sensor on pin 4
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"Sensor reading: {temp:.2f} °C at {timestamp}")  # Debug
            with open(LOG_FILE, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, f"{temp:.2f}"])
            mqtt_client.publish(MQTT_TOPIC_SENSOR, f"Temperature:{temp:.2f}")
            st.session_state['latest_temp'] = temp
            st.session_state['temp_data'].append({'Timestamp': timestamp, 'Temperature': temp})
            if len(st.session_state['temp_data']) > 50:
                st.session_state['temp_data'].pop(0)
        except Exception as e:
            print(f"Error in sensor loop: {e}")
        time.sleep(5)

# Simulated GPIO setup
def setup_gpio():
    GPIO.setmode('BCM')
    for device in devices:
        GPIO.setup(devices[device]['pin'], GPIO.OUTPUT)
    GPIO.setup(4, GPIO.OUTPUT)  # Sensor pin
    print("GPIO setup complete")  # Debug

# Streamlit app
def main():
    st.set_page_config(page_title="Smart Home Control", layout="wide")
    st.title("Smart Home Control System")

    # Initialize session state
    if 'temp_data' not in st.session_state:
        st.session_state['temp_data'] = []
    if 'latest_temp' not in st.session_state:
        st.session_state['latest_temp'] = 0.0
    for device in devices:
        if f"{devices[device]['pin']}_state" not in st.session_state:
            st.session_state[f"{devices[device]['pin']}_state"] = GPIO.LOW

    # Force periodic rerender
    if 'last_update' not in st.session_state:
        st.session_state['last_update'] = time.time()
    if time.time() - st.session_state['last_update'] > 5:
        st.session_state['last_update'] = time.time()
        st.rerun()  # Refresh UI every 5 seconds

    # Layout
    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Device Controls")
        for device, info in devices.items():
            st.write(f"**{info['name']}**")
            toggle = st.checkbox(
                label=f"Toggle {info['name']}",
                value=info['state'] == GPIO.HIGH,
                key=f"checkbox_{device}"
            )
            if toggle != (info['state'] == GPIO.HIGH):
                state = GPIO.HIGH if toggle else GPIO.LOW
                devices[device]['state'] = state
                GPIO.output(info['pin'], state)
                mqtt_client.publish(MQTT_TOPIC_CONTROL, f"{device}:{'ON' if state == GPIO.HIGH else 'OFF'}")
                log_action(device, 'ON' if state == GPIO.HIGH else 'OFF')

        st.subheader("Temperature Sensor")
        st.metric(label="Current Temperature", value=f"{st.session_state['latest_temp']:.2f} °C")

    with col2:
        st.subheader("Temperature History")
        if st.session_state['temp_data']:
            df = pd.DataFrame(st.session_state['temp_data'])
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            st.line_chart(df.set_index('Timestamp')['Temperature'])

        st.subheader("Recent Actions")
        if os.path.exists(ACTION_LOG_FILE):
            df_actions = pd.read_csv(ACTION_LOG_FILE)
            st.dataframe(df_actions.tail(10))

    # Start background threads
    if 'mqtt_thread' not in st.session_state:
        setup_gpio()
        st.session_state['mqtt_thread'] = Thread(target=start_mqtt)
        st.session_state['mqtt_thread'].start()
        st.session_state['sensor_thread'] = Thread(target=sensor_loop)
        st.session_state['sensor_thread'].start()

if __name__ == "__main__":
    main()